In [ ]:
# importing modules and packages 
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt
import datetime
import seaborn as sns
from sklearn import model_selection
from sklearn.model_selection import train_test_split 
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LogisticRegression
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.metrics import mean_squared_error, mean_absolute_error 
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler as Scaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
seed = 101
# import all libraries needed

In [ ]:
df = pd.read_csv("data/actual.csv")
df.head()

In [ ]:
df = df[df["Actual_Price"]>= 50]

In [ ]:
df.isnull().sum()

In [ ]:
df["price_date"] = pd.to_datetime(df["price_date"], format="%m/%d/%Y")

In [ ]:
#df = df.sort_values('price_date', ascending=False)

In [ ]:
df["price_date"] = df["price_date"].dt.strftime('%Y-%m-%d')
df.head()

In [ ]:
df0 = df.copy()

In [ ]:
#converting string columns to numerical
le = preprocessing.LabelEncoder()
df["price_date"] = le.fit_transform(df["price_date"])
df["parish"] = le.fit_transform(df["parish"])
df["commodity"] = le.fit_transform(df["commodity"])
df["type"] = le.fit_transform(df["type"])
df["Category"] = le.fit_transform(df["Category"])
df["supply"] = le.fit_transform(df["supply"])
df["grade"] = le.fit_transform(df["grade"])

In [ ]:
# apply normalization techniques by Column 1 
#column = 'Actual_Price'
#df[column] = (df[column] - df[column].min()) / (df[column].max() - df[column].min())  

In [ ]:
df.to_csv("before.csv")

In [ ]:
x = df.values #returns a numpy array
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)
df_scaled = pd.DataFrame(x_scaled, columns = ['price_date', 'parish', 'commodity', 'type', 'Category', 'supply', 'grade', 'Actual_Price'] )
df= df_scaled.copy()

In [ ]:
#normalized_df=(df-df.min())/(df.max()-df.min())
#df = normalized_df.copy()

In [ ]:
df.head()

In [ ]:
#df = normalized_df.copy()

In [ ]:
#using One Hot encoder to convert string columns into numerical columns
#df_encoded = pd.get_dummies(df, columns=['parish', 'commodity', 'type', 'Category', 'supply', 'grade'])
#df_encoded.head()

In [ ]:
#df = df_encoded.copy()

In [ ]:
d = dict.fromkeys(df.select_dtypes(np.int64).columns, np.int32)

In [ ]:
df = df.astype(d)

In [ ]:
df['Actual_Price'] = df['Actual_Price'].astype(float)
df.info()

In [ ]:
df.to_csv('test.csv')

In [ ]:
corr = df.corr()
corr

In [ ]:
#sns.heatmap(corr, xticklabels=corr.columns, yticklabels=corr.columns)

In [ ]:
df.hist(bins=50, figsize=(25,10))
plt.show()

In [ ]:
df.columns

In [ ]:
X = np.array(df.drop(['Actual_Price'], axis=1))
y = np.array(df['Actual_Price'])

In [ ]:
seed = 42
scoring = 'r2'

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3, random_state=seed)

In [ ]:
#scaling x
#mm_scaler = Scaler()
#mm_scaler = preprocessing.MinMaxScaler(feature_range=(0,1))
#mm_scaler.fit(X_train)
#X_train = mm_scaler.transform(X_train)
#X_test = mm_scaler.transform(X_test)

In [ ]:
# Spot Check Algorithms
models = []
models.append(('LNR', LinearRegression()))
models.append(('LDA', LinearDiscriminantAnalysis()))
models.append(('KNN', KNeighborsClassifier()))
models.append(('CART', DecisionTreeClassifier()))
models.append(('RGR', RandomForestRegressor()))
#models.append(('NB', GaussianNB()))
models.append(('SVM', SVC()))

In [ ]:
# evaluate each model in turn
results = []
names = []
for name, model in models:
    kfold = model_selection.KFold(n_splits=10, random_state=None)
    cv_results = model_selection.cross_val_score(model, X_train, y_train, cv=kfold, scoring='r2')
    results.append(cv_results)
    names.append(name)
    msg = "%s: %f (%f)" % (name, cv_results.mean(), cv_results.std())
    print(msg)

In [ ]:
#tuned parameters for random forest regressor
from sklearn.model_selection import GridSearchCV
tuned_parameters = {
    'max_depth' : [2,3,5,10,20],
    'min_samples_leaf': [5,10,20,50,100,200],        
    'n_estimators': [10,25,30,50,100,200]   
}
rgr_model = RandomForestRegressor()
gridSearch = GridSearchCV(rgr_model, tuned_parameters, cv=10, scoring='r2')
gridSearch.fit(X_train, y_train)

In [ ]:
"""
from sklearn.model_selection import GridSearchCV
tuned_parameters = {
    'solver' : ['svd', 'lsqr', 'eigen'],
    'shrinkage': ['auto', 0.1, 0.5],        
    'tol' : [0.1, 0.3, 0.5]   
}
lda_model = LinearDiscriminantAnalysis()
gridSearch = GridSearchCV(lda_model, tuned_parameters, cv=10, scoring='r2')
gridSearch.fit(X_train, y_train)"""

In [ ]:
gridSearch.best_score_

In [ ]:
gridSearch.best_params_

In [ ]:
rgr_model = gridSearch.best_estimator_
print(rgr_model)

In [ ]:
rgr_model.fit(X_train, y_train)

In [ ]:
rgr_model.oob_score_

In [ ]:
# making predictions 
predictions = rgr_model.predict(X_test)

In [ ]:
# model evaluation 
print('mean_squared_error : ', mean_squared_error(y_test, predictions)) 
print('mean_absolute_error : ', mean_absolute_error(y_test, predictions))

In [ ]:
total_predictions = rgr_model.predict(X)
total_predictions

In [ ]:
df0["Predictions"] = total_predictions
df0.head()

In [ ]:
#denormalizinf predictions
#column = "Predictions"
#df0["Predictions"] = (df0[column] * (df0[column].max() - df0[column].min()) + df0[column].min())
#df0.head()

In [ ]:
df0.to_csv("predictions.csv")

### Linear Discriminate Analysis


In [ ]:
#tuned parameters for Linear DiscriminateAnalysis
from sklearn.model_selection import GridSearchCV
tuned_parameters = {
    'solver' : ['svd', 'lsqr', 'eigen'],
    'min_samples_split': [2, 4],        
    'max_features' : [None]   
}
rgr_model = RandomForestRegressor()
gridSearch = GridSearchCV(rgr_model, tuned_parameters, cv=10, scoring='r2')
gridSearch.fit(X_train, y_train)

In [ ]:
# making predictions 
predictions = rgr.predict(X_test)

In [ ]:
# model evaluation 
print('mean_squared_error : ', mean_squared_error(y_test, predictions)) 
print('mean_absolute_error : ', mean_absolute_error(y_test, predictions))

In [ ]:
#model = LinearRegression()
model = RandomForestRegressor()

In [ ]:
# fitting the model 
model.fit(X_train_scaled, y_train)

In [ ]:
# making predictions 
predictions = model.predict(X_test_scaled)

In [ ]:
# model evaluation 
print('mean_squared_error : ', mean_squared_error(y_test, predictions)) 
print('mean_absolute_error : ', mean_absolute_error(y_test, predictions))

In [ ]:
mm_scaler = Scaler()
mm_scaler.fit(X)
X_full = mm_scaler.transform(X)

In [ ]:
total_predictions = model.predict(X_full)

In [ ]:
df0["Predictions"] = total_predictions
df0.head()

In [ ]:
import pickle

filename = "farmgate_model.pickle"

#save model
pickle.dump(model, open(filename, "wb"))

# load model
loaded_model = pickle.load(open(filename, "rb"))

In [ ]:
#filterting for unique rows without dates

In [ ]:
df = pd.read_csv("actual.csv")
df = df.drop(["price_date", "Actual_Price"], axis =1)
df.head()

In [ ]:
dff = df.drop_duplicates()
dff.shape

In [ ]:
#creating a list of dates for next three years
base = datetime.datetime.today()
#date_list = [base + datetime.timedelta(days=x) for x in range(numdays)]

In [ ]:
base
